#### Importing Libraries

In [15]:
import pandas as pd
from sklearn.model_selection import train_test_split
import string
from string import digits
import re
from sklearn.utils import shuffle
from keras.layers import LSTM, Input, Dense,Embedding
from keras.models import Model,load_model
from keras.preprocessing.text import one_hot
from keras.preprocessing.text import Tokenizer
from keras.models import model_from_json
import pickle as pkl
import numpy as np
from keras.utils.vis_utils import plot_model
import matplotlib.pyplot as plt
from tensorflow import keras
pad_sequences = keras.preprocessing.sequence.pad_sequences

### Download the data

This is a shell command that downloads a zip file containing French-English data from a website and extracts its contents using the unzip utility. The first command downloads the file, and the second command extracts it into the current directory.

In [16]:
!!curl -O http://www.manythings.org/anki/fra-eng.zip
!!unzip fra-eng.zip

['Archive:  fra-eng.zip',
 'replace _about.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename:  NULL',
 '(EOF or read error, treating as "[N]one" ...)']

### Prepare the data

This function reads text data from a file and processes it to extract input and target texts, and creates sets of input and target characters. It returns these data for further processing. The input and target texts are stored in separate lists, while the input and target characters are stored in separate sets. The function also adds a start sequence character and an end sequence character to the target texts.

In [17]:
# Path to the data txt file on disk.
data_path = "fra.txt"

num_samples = 20000  # Number of samples to train on.

# Vectorize the data.
def read_and_vectorize_data(data_path):
    input_texts = []
    target_texts = []
    input_characters = set()
    target_characters = set()

    with open(data_path, "r", encoding="utf-8") as f:
        lines = f.read().split("\n")

    for line in lines[: min(num_samples, len(lines) - 1)]:
        input_text, target_text, _ = line.split("\t")
        # We use "tab" as the "start sequence" character
        # for the targets, and "\n" as "end sequence" character.
        target_text = "\t" + target_text + "\n"
        input_texts.append(input_text)
        target_texts.append(target_text)
        
        input_text = set(input_text)
        input_characters = input_characters.union(input_text)
        
        target_text = set(target_text)
        target_characters = target_characters.union(target_text)
            
    return input_texts, target_texts, input_characters, target_characters

This code processes the output of read_and_vectorize_data by computing statistics and generating dictionaries for the input and target characters. It prints information about the data, including the number of samples, the number of unique input and output tokens, and the maximum sequence length for inputs and outputs. It also generates dictionaries for the input and target characters using dictionary comprehension, which will be used later to convert the input and target texts to numerical sequences.

In [18]:
input_texts, target_texts, input_characters, target_characters = read_and_vectorize_data(
    data_path=data_path
)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)

# Generate dictionary for characters
input_token_index = {char: i for i, char in enumerate(input_characters)}
target_token_index = {char: i for i, char in enumerate(target_characters)}

Number of samples: 20000
Number of unique input tokens: 74
Number of unique output tokens: 100
Max sequence length for inputs: 17
Max sequence length for outputs: 59


This function creates input/target data arrays for the model. It initializes three numpy arrays with zeros, and then loops through each input/target text pair. It fills in the encoder_input_data array with one-hot encoded characters from the input text. It fills in the decoder_input_data and decoder_target_data arrays with one-hot encoded characters from the target text, where the decoder_target_data array is shifted by one character to predict the next character.

In [19]:
# Prepare input/target data for model
def create_data_arrays():
    encoder_input_data = np.zeros(
        (len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32"
    )
    decoder_input_data = np.zeros(
        (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
    )
    decoder_target_data = np.zeros(
        (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
    )

    for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
        for t, char in enumerate(input_text):
            encoder_input_data[i, t, input_token_index[char]] = 1.0
        encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0

        for t, char in enumerate(target_text):
            # decoder_target_data is ahead of decoder_input_data by one timestep
            decoder_input_data[i, t, target_token_index[char]] = 1.0
            if t > 0:
                # decoder_target_data will be ahead by one timestep
                # and will not include the start character.
                decoder_target_data[i, t - 1, target_token_index[char]] = 1.0

        decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
        decoder_target_data[i, t:, target_token_index[" "]] = 1.0
        
    return encoder_input_data, decoder_input_data, decoder_target_data


In [20]:
encoder_input_data, decoder_input_data, decoder_target_data = create_data_arrays()

### Build the model

This function builds a sequence-to-sequence model using an LSTM encoder-decoder architecture with one-hot encoding. The encoder inputs are processed by an LSTM layer, and its output states are passed to the decoder. The decoder inputs are also passed through an LSTM layer with its own internal states. The final decoder output is processed by a dense layer with softmax activation to produce the predicted output sequence. The model is compiled using the RMSprop optimizer and categorical cross-entropy loss function.

In [21]:
def build_model(
    num_encoder_tokens, num_decoder_tokens, latent_dim=128, mname='char_model'
):
    # Define an input sequence and process it.
    encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))
    encoder = keras.layers.LSTM(latent_dim, return_state=True)
    encoder_outputs, state_h, state_c = encoder(encoder_inputs)

    # We discard `encoder_outputs` and only keep the states.
    encoder_states = [state_h, state_c]

    # Set up the decoder, using `encoder_states` as initial state.
    decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))

    # We set up our decoder to return full output sequences,
    # and to return internal states as well. We don't use the
    # return states in the training model, but we will use them in inference.
    decoder_lstm = keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True)
    decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
    decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax")
    decoder_outputs = decoder_dense(decoder_outputs)

    # Define the model that will turn
    # `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
    model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs, name=mname)
    
    model.compile(
        optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"]
    )
    
    return model

### Train the model

This code builds a model with a 256-dimensional LSTM encoder and decoder, and prints a summary of the model architecture.

In [22]:
model = build_model(
    num_encoder_tokens=num_encoder_tokens,
    num_decoder_tokens=num_decoder_tokens,
    latent_dim=256,  # Latent dimensionality of the encoding space.
    mname='char_model'
)
model.summary()

Model: "char_model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, None, 74)]   0           []                               
                                                                                                  
 input_6 (InputLayer)           [(None, None, 100)]  0           []                               
                                                                                                  
 lstm_2 (LSTM)                  [(None, 256),        338944      ['input_5[0][0]']                
                                 (None, 256),                                                     
                                 (None, 256)]                                                     
                                                                                         

This code trains the model using the input and target data arrays. It specifies the batch size and number of epochs, then calls the fit method on the model with the data arrays and training parameters as inputs. The model is also saved to disk with the name "s2s".

In [24]:
batch_size = 64  # Batch size 
epochs = 100  # Number of epochs

history = model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2,
)
# Save model
model.save("s2s")

Epoch 1/100
250/250 [==============================] - 9s 38ms/step - loss: 0.1327 - accuracy: 0.9576 - val_loss: 0.5220 - val_accuracy: 0.8788
Epoch 2/100
250/250 [==============================] - 9s 35ms/step - loss: 0.1309 - accuracy: 0.9583 - val_loss: 0.5296 - val_accuracy: 0.8779
Epoch 3/100
250/250 [==============================] - 9s 37ms/step - loss: 0.1292 - accuracy: 0.9586 - val_loss: 0.5314 - val_accuracy: 0.8788
Epoch 4/100
250/250 [==============================] - 10s 38ms/step - loss: 0.1277 - accuracy: 0.9591 - val_loss: 0.5329 - val_accuracy: 0.8784
Epoch 5/100
250/250 [==============================] - 10s 40ms/step - loss: 0.1258 - accuracy: 0.9596 - val_loss: 0.5364 - val_accuracy: 0.8788
Epoch 6/100
250/250 [==============================] - 10s 41ms/step - loss: 0.1243 - accuracy: 0.9600 - val_loss: 0.5397 - val_accuracy: 0.8791
Epoch 7/100
250/250 [==============================] - 11s 45ms/step - loss: 0.1226 - accuracy: 0.9604 - val_loss: 0.5438 - val_accur

INFO:tensorflow:Assets written to: s2s/assets


INFO:tensorflow:Assets written to: s2s/assets



### Run inference (sampling)

This code defines two sampling models to be used during inference, based on the previously trained model. The encoder model takes input text and returns the encoder states, while the decoder model takes in the decoder inputs and initial decoder states, and returns the predicted outputs and updated decoder states.

In [27]:
# Define sampling models
# Restore the model and construct the encoder and decoder.
def load_saved_model(model_ckpt="s2s",latent_dim=256):
    model = keras.models.load_model(model_ckpt)

    encoder_inputs = model.input[0]  # input_1
    encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output  # lstm_1
    encoder_states = [state_h_enc, state_c_enc]
    encoder_model = keras.Model(encoder_inputs, encoder_states)

    decoder_inputs = model.input[1]  # input_2
    decoder_state_input_h = keras.Input(shape=(latent_dim,))
    decoder_state_input_c = keras.Input(shape=(latent_dim,))
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    decoder_lstm = model.layers[3]
    decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
        decoder_inputs, initial_state=decoder_states_inputs
    )
    decoder_states = [state_h_dec, state_c_dec]
    decoder_dense = model.layers[4]
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = keras.Model(
        [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
    )
    return encoder_model, decoder_model

In [28]:
encoder_model, decoder_model = load_saved_model()

This code defines a function to decode a sequence using the encoder and decoder models. The input sequence is first encoded by the encoder model to get the initial states. The function then uses a loop to generate the output sequence one character at a time using the decoder model and the previous output as input. The loop terminates when the end-of-sequence character is generated or the maximum sequence length is reached. Finally, the function returns the decoded sequence as a string.

In [29]:
# Reverse-lookup token index to decode sequences back to something readable.
reverse_input_char_index = {i: char for char, i in input_token_index.items()}
reverse_target_char_index = {i: char for char, i in target_token_index.items()}

def decode_sequence(input_seq, encoder_model, decoder_model):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index["\t"]] = 1.0

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        # Update states
        states_value = [h, c]
    return decoded_sentence

This code generates translations using the trained encoder and decoder models. It takes an input sentence, encodes it with the encoder model, decodes it with the decoder model, and prints the original, decoded, and expected sentences.

In [30]:
for seq_index in range(25, 30):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(input_seq, encoder_model, decoder_model)
    print("-")
    print("Input sentence:", input_texts[seq_index])
    print("Decoded sentence:", decoded_sentence)
    print("Original sentence:", target_texts[seq_index])

1/1 [==============================] - 0s 10ms/step
-
Input sentence: Wow!
Decoded sentence: Ça alors !

Original sentence: 	Wah !

1/1 [==============================] - 0s 11ms/step
-
Input sentence: Duck!
Decoded sentence: Baisse-toi !

Original sentence: 	À terre !

1/1 [==============================] - 0s 10ms/step
-
Input sentence: Duck!
Decoded sentence: Baisse-toi !

Original sentence: 	Baisse-toi !

1/1 [==============================] - 0s 10ms/step
-
Input sentence: Duck!
Decoded sentence: Baisse-toi !

Original sentence: 	Baissez-vous !

1/1 [==============================] - 0s 11ms/step
-
Input sentence: Fire!
Decoded sentence: Au feu !

Original sentence: 	Au feu !

